# IESB - Graduacao - CIA035 - MNIST CNN

In [1]:
# Importando as bibliotecas
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch import nn, optim

In [2]:
# Dterminando se podemos trabalhar com GPU
device = ("cuda" if torch.cuda.is_available() else "cpu")

device

'cuda'

In [3]:
# As tranformações que serão aplicadas a cada imagem
# No caso elas serão convertidas em tensores e normalizadas
# Para o MNIST o valor da média é 0.1307 e o desvio padrão é 0.3081
image_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.1307,), (0.3081,))])

In [4]:
# Baixando o dataset de treino e teste
# Aplica as transformações nas imagens de treino e de teste
# Os dados são carregados junto com seus labels (o nome de cada pasta)
traindata = datasets.MNIST('data', download=True, train=True, transform=image_transform)
testdata = datasets.MNIST('data', download=True, train=False, transform=image_transform)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629403081/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# Carregando as imagens de treino e teste
# O batch_size determina o tamanho do conjunto de dados a ser aplicado a rede
trainloader = torch.utils.data.DataLoader(traindata, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testdata, batch_size=64, shuffle=True)

In [6]:
# Definindo uma rede neural convolucional

# Cálculo do tamanho da saída de cada convolução
# outputOfEachConvLayer = [(size + 2*padding - kernel_size) / stride] + 1

# No MNIST as imagens só possuem 1 canal de entrada (não é RGB)
model = nn.Sequential(
                      # Parte de convolução
                      nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3, padding=1, stride=1),
                      # [(28 + 2*1 - 3)/1] + 1 = 28
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2),
                      # 28/2 = 14
                      
                      nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, padding=1, stride=1),
                      # [(14 + 2*1 - 3)/1] + 1 = 14
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2),
                      # 14/2 = 7
                      
                      # Convertendo a imagem em estrutura plana
                      nn.Flatten(),
                      
                      # Parte fully connected
                      nn.Linear(in_features=24*7*7, out_features=64),
                      # in_features = 1176 
                      nn.ReLU(),
                      nn.Dropout(p=0.2), # Dropout com probabilidade de 0.2
                      nn.Linear(in_features=64, out_features=10))

model.to(device)

Sequential(
  (0): Conv2d(1, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten()
  (7): Linear(in_features=1176, out_features=64, bias=True)
  (8): ReLU()
  (9): Dropout(p=0.2, inplace=False)
  (10): Linear(in_features=64, out_features=10, bias=True)
)

In [7]:
# Temos que definir a função de erro e o otimizador (que vai alterar os pesos dos perceptrons)
error_function = nn.CrossEntropyLoss() # criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

error_function.to(device)

CrossEntropyLoss()

In [8]:
# Treinamento do modelo

# Definindo o número de épocas
epochs = 10

# Colocando o modelo em modo de treinamento
model.train()


# For para rodar o número de épocas
for i in range(epochs):
    # Treinamento
    
    # Monitorando o training loss
    train_loss = 0.0
    
    # Obtendo dados e respostas
    for data, target in trainloader:
        
        # Enviando os dados para GPU, se existir
        data, target = data.to(device), target.to(device)
    
        # Foward Propagation (passando os dados de treino pela rede)
        outputs = model(data)
        # Calculando o erro
        loss = error_function(outputs, target)
       
        # Back Propagation
        # Limpar os parametros do otimizador (zerar o Gradiente Descendent)
        optimizer.zero_grad()
        # Calcular os novos pesos
        loss.backward()
        # Executar o optimizador (efetivamente fazer o back propagation mudando os pesos)
        optimizer.step()
        
        # Atualizando o training loss
        train_loss += loss.item() * data.size(0)
        
    # Calculando a média de erro por epoch
    train_loss = train_loss/len(trainloader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(i+1, train_loss))

Epoch: 1 	Training Loss: 0.292668
Epoch: 2 	Training Loss: 0.091467
Epoch: 3 	Training Loss: 0.065626
Epoch: 4 	Training Loss: 0.052684
Epoch: 5 	Training Loss: 0.044650
Epoch: 6 	Training Loss: 0.037164
Epoch: 7 	Training Loss: 0.032869
Epoch: 8 	Training Loss: 0.030658
Epoch: 9 	Training Loss: 0.027740
Epoch: 10 	Training Loss: 0.025803


In [9]:
# Variaveis para controlar os acertos das previsões da rede
# e  calcular a acurácia
correct = 0
total = 0

# Vamos colocar o modelo em modo de avaliação/teste
model.eval()

# Obtendo dados e respostas
for data, target in testloader:
    
    # Enviando os dados para GPU, se existir
    data, target = data.to(device), target.to(device)    
    
    output = model(data)
    
    for index, i in enumerate(output):
        if torch.argmax(i) == target[index]:
            correct += 1
        total += 1

In [10]:
print('Accuracy: ', round(correct/total, 3))

Accuracy:  0.991
